# Downloading the Law HTML Page / Descargando la página HTML de la ley

**Goal**: Download the Paraguay Labor Code HTML page and save it locally for further processing.  

**Input**: The URL of the law page on the official government website.  

**Approach**:  
- Use the `requests` library to perform an HTTP GET request.  
- Ensure the request is successful and handle possible errors.  
- Create the target folder if it does not exist.  
- Save the HTML content to a file at `../data/raw/codigo_trabajo_py.html`.

**Output**: Local HTML file containing the law text.  

**Why this matters**: Having a local copy ensures consistent processing and avoids repeated web requests, which is especially useful for parsing, indexing, or text analysis tasks.

---

**Objetivo**: Descargar la página HTML del Código Laboral de Paraguay y guardarla localmente para su posterior procesamiento.  

**Entrada**: URL de la página de la ley en el sitio oficial del gobierno.  

**Enfoque**:  
- Usar la librería `requests` para hacer una petición HTTP GET.  
- Verificar que la petición sea exitosa y manejar posibles errores.  
- Crear la carpeta de destino si no existe.  
- Guardar el contenido HTML en un archivo en `../data/raw/codigo_trabajo_py.html`.

**Salida**: Archivo HTML local que contiene el texto de la ley.  

**Por qué importa**: Tener una copia local garantiza un procesamiento consistente y evita solicitudes web repetidas, útil para parsing, indexación o análisis de texto.


In [26]:
import requests
from pathlib import Path

def download_law_page(url, output_path = "../data/raw/codigo_trabajo_py.html"):
    out_path = Path(output_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    
    response = requests.get(url)
    response.raise_for_status()  # Lanza excepción si hay error en la descarga
    
    # Guardar contenido en archivo
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(response.text)
    
    print(f"Página descargada y guardada en: {out_path}")

url = "https://www.bacn.gov.py/leyes-paraguayas/2608/ley-n-213-establece-el-codigo-del-trabajo"
download_law_page(url)

Página descargada y guardada en: ../data/raw/codigo_trabajo_py.html


# Transformation of downloaded data / Transformación de los datos descargados

**Goal**: Extract the Paraguay Labor Code text from a local HTML file (`../data/raw/codigo_trabajo_py.html`) for further processing, such as cleaning, indexing, or search.  

**Input**: Local HTML file saved from the source website.  

**Approach**:  
- Open the file using a context manager with the selected encoding.  
- Parse the HTML with BeautifulSoup using the built-in `html.parser`.  
- Locate the main container `<div class="entry-content">`.  
- Extract the text while preserving line breaks.

**Output**: Clean plain text in memory.  

**Why this matters**: Ensures that the extracted text is complete and readable, making it suitable for downstream tasks like indexing, search, or QA.

**File path and encoding**

**Path**: `../data/raw/codigo_trabajo_py.html`. Adjust if your working directory changes.  

**Encoding**: The file is likely in Latin-1 (`encoding='latin-1'`). If you notice encoding artifacts such as `Ã³`, try `utf-8` or `cp1252`.  

**Reading the HTML**

Use a context manager to safely open and read the file.  

**Common pitfalls**:  
- FileNotFoundError if the path is wrong.  
- UnicodeDecodeError if the encoding does not match the file.

**Parse and select content**

**Parser**: `html.parser` is built-in and sufficient for this HTML.  

**Target container**: The law text is inside `<div class="entry-content">`, selected with `soup.find('div', class_='entry-content')`.  

**Fallbacks**: If `None` is returned, inspect the DOM for alternative classes or IDs.

**Extract text cleanly**

Use `get_text(separator='\n', strip=True)` to flatten the HTML while preserving line breaks.  

**Tips**: Adjust `separator` for paragraph spacing or post-process the text to normalize multiple newlines or bullet points.

---

**Objetivo**: Extraer el texto del Código Laboral de Paraguay desde un archivo HTML local (`../data/raw/codigo_trabajo_py.html`) para procesamiento posterior, como limpieza, indexación o búsqueda.  

**Entrada**: Archivo HTML local guardado desde el sitio de origen.  

**Enfoque**:  
- Abrir el archivo con un gestor de contexto usando la codificación correcta.  
- Parsear el HTML con BeautifulSoup (`html.parser`).  
- Localizar el contenedor principal `<div class="entry-content">`.  
- Extraer el texto preservando los saltos de línea.

**Salida**: Texto plano limpio en memoria.  

**Por qué importa**: Garantiza que el texto extraído esté completo y sea legible, apto para tareas posteriores como indexación, búsqueda o QA.

**Ruta y codificación**

**Ruta**: `../data/raw/codigo_trabajo_py.html`. Ajusta si cambia tu directorio de trabajo.  

**Codificación**: Probablemente el archivo está en Latin-1 (`encoding='latin-1'`). Si aparecen artefactos como `Ã³`, prueba con `utf-8` o `cp1252`.

**Lectura del HTML**

Usa un gestor de contexto para abrir y leer el archivo de forma segura.  

**Errores comunes**:  
- FileNotFoundError si la ruta es incorrecta.  
- UnicodeDecodeError si la codificación no coincide con el archivo.

**Parseo y selección de contenido**

**Parser**: `html.parser` es integrado y suficiente para este HTML.  

**Contenedor objetivo**: El texto de la ley está dentro de `<div class="entry-content">`, seleccionado con `soup.find('div', class_='entry-content')`.  
**Alternativas**: Si retorna `None`, inspecciona el DOM para otras clases o IDs posibles.

**Extracción de texto limpio**

Usa `get_text(separator='\n', strip=True)` para aplanar el HTML preservando los saltos de línea.  

**Consejos**: Ajusta `separator` para espaciar párrafos o post-procesa el texto para normalizar saltos de línea múltiples o viñetas.


In [27]:
import re
import json
from pathlib import Path
from ftfy import fix_text
from bs4 import BeautifulSoup

In [2]:
nombre_archivo_html = '../data/raw/codigo_trabajo_py.html'

with open(nombre_archivo_html, 'r', encoding='latin-1') as archivo:
    contenido_html = archivo.read()

In [4]:
soup = BeautifulSoup(contenido_html, 'html.parser')
contenido_ley = soup.find('div', class_='entry-content')

In [6]:
if contenido_ley:
    texto_limpio = contenido_ley.get_text(separator='\n', strip=True)
    print("--- Contenido de la Ley extraído exitosamente ---")
else:
    print("Error: No se pudo encontrar el contenedor del contenido de la ley (div con clase 'entry-content').")

In [20]:
#print(texto_limpio)

# Header and Article Patterns / Patrones de Encabezados y Artículos

This section defines regular expressions and mappings to identify different parts of the legal text:

- `HEADER_PATTERNS`: A dictionary containing regex patterns for:
  - `'libro'`: Matches "LIBRO" followed by its name (e.g., "LIBRO PRIMERO").
  - `'titulo'`: Matches "TITULO" followed by its name (e.g., "TITULO PRIMERO").
  - `'capitulo'`: Matches "CAPITULO" followed by Roman numerals (e.g., "CAPITULO I").
  
- `ARTICULO_PATTERN`: A regex to detect article headers like "Artículo 1°.-".

- `ROMAN_MAP`: Maps Spanish ordinal words to integer numbers for easy conversion (e.g., "PRIMERO" → 1).

---

Esta sección define expresiones regulares y mapeos para identificar diferentes partes del texto legal:

- `HEADER_PATTERNS`: Un diccionario con patrones regex para:
  - `'libro'`: Detecta "LIBRO" seguido de su nombre (ej.: "LIBRO PRIMERO").
  - `'titulo'`: Detecta "TITULO" seguido de su nombre (ej.: "TITULO PRIMERO").
  - `'capitulo'`: Detecta "CAPITULO" seguido de números romanos (ej.: "CAPITULO I").
  
- `ARTICULO_PATTERN`: Regex para detectar encabezados de artículos como "Artículo 1°.-".

- `ROMAN_MAP`: Mapea palabras ordinales en español a números enteros para facilitar la conversión (ej.: "PRIMERO" → 1).


In [9]:
HEADER_PATTERNS = {
    'libro': re.compile(r"^LIBRO\s+([A-ZÁÉÍÓÚÑ]+)\s*$", re.IGNORECASE),
    'titulo': re.compile(r"^TITULO\s+([A-ZÁÉÍÓÚÑ]+)\s*$", re.IGNORECASE),
    'capitulo': re.compile(r"^CAPITULO\s+([IVXLCDM]+)\s*$", re.IGNORECASE),
}

ARTICULO_PATTERN = re.compile(r"^Art[íi]?t?culo\s+(\d+)\s*(?:[°º])?\s*\.?\s*-\s*$", re.IGNORECASE)

ROMAN_MAP = {
    'PRIMERO': 1, 'SEGUNDO': 2, 'TERCERO': 3, 'CUARTO': 4, 'QUINTO': 5,
    'SEXTO': 6, 'SÉPTIMO': 7, 'SEPTIMO': 7, 'OCTAVO': 8, 'NOVENO': 9,
    'DÉCIMO': 10, 'DECIMO': 10, 'UNDÉCIMO': 11, 'UNDECIMO': 11, 'DUODÉCIMO': 12, 'DUODECIMO': 12,
}

# Roman Numeral Conversion / Conversión de Números Romanos

This section defines a helper function to convert Roman numerals into integers:

- `_ROMAN_VALUES`: A dictionary mapping Roman numeral characters to their integer values.
- `roman_to_int(roman)`: Converts a Roman numeral string into an integer.
  - The function iterates over the characters in reverse.
  - If a smaller value precedes a larger one, it is subtracted.
  - Otherwise, the value is added.

---

Esta sección define una función auxiliar para convertir números romanos en enteros:

- `_ROMAN_VALUES`: Un diccionario que asigna valores enteros a los caracteres de números romanos.
- `roman_to_int(roman)`: Convierte una cadena de número romano a un número entero.
  - La función itera sobre los caracteres en orden inverso.
  - Si un valor menor precede a un valor mayor, se resta.
  - De lo contrario, se suma el valor.


In [10]:
_ROMAN_VALUES = {"I":1,"V":5,"X":10,"L":50,"C":100,"D":500,"M":1000}

def roman_to_int(roman):
    roman = roman.strip().upper()
    total = 0
    prev = 0
    for ch in reversed(roman):
        val = _ROMAN_VALUES.get(ch, 0)
        if val < prev:
            total -= val
        else:
            total += val
            prev = val
    return total

# Metadata Extraction / Extracción de Metadatos

This function extracts key metadata from the legal text header, before the first chapter:

- `extract_metadata(lines)`: Receives a list of text lines.
  - Constructs the `encabezado` (header) until the first "CAPÍTULO I".
  - Searches within this header for:
    - Law number (`numero_ley`) using a regex that matches "LEY N° ...".
    - Promulgation date (`fecha_promulgacion`) using a regex for "Fecha de Promulgación".
    - Publication date (`fecha_publicacion`) using a regex for "Fecha de Publicación".
  - Returns a dictionary `meta` containing these values.

---

Esta función extrae metadatos clave del encabezado del texto legal, antes del primer capítulo:

- `extract_metadata(lines)`: Recibe una lista de líneas de texto.
  - Construye el `encabezado` hasta el primer "CAPÍTULO I".
  - Busca dentro de este encabezado:
    - Número de ley (`numero_ley`) usando una expresión regular que detecta "LEY N° ...".
    - Fecha de promulgación (`fecha_promulgacion`) usando regex para "Fecha de Promulgación".
    - Fecha de publicación (`fecha_publicacion`) usando regex para "Fecha de Publicación".
  - Devuelve un diccionario `meta` con estos valores.


In [11]:
def extract_metadata(lines):
    """Extrae número de ley y fechas desde el encabezado (antes del primer CAPITULO I)."""
    meta = {}

    encabezado = []
    for ln in lines:
        if re.match(r"CAP[IÍ]TULO\s+I\b", ln, re.IGNORECASE):
            break
        encabezado.append(ln)

    encabezado_text = " ".join(encabezado)

    ley_match = re.search(r"LEY\s*N[°º]?\s*(\d+)", encabezado_text, re.IGNORECASE)
    if ley_match:
        meta["numero_ley"] = ley_match.group(1)

    promulg_match = re.search(r"Fecha\s+de\s+Promulgaci[oó]n:?\s*(\d{2}-\d{2}-\d{4})", encabezado_text, re.IGNORECASE)
    if promulg_match:
        meta["fecha_promulgacion"] = promulg_match.group(1)


    public_match = re.search(r"Fecha\s+de\s+Publicaci[oó]n:?\s*(\d{2}-\d{2}-\d{4})", encabezado_text, re.IGNORECASE)
    if public_match:
        meta["fecha_publicacion"] = public_match.group(1)

    return meta

# Article Extraction / Extracción de Artículos

This function segments the legal text into structured parts: books, titles, chapters, and articles in detail.

- `extract_articles(lines)`: Receives a list of text lines.
  - Maintains context variables for the current book, title, chapter, and chapter description.
  - Uses the helper function `flush_article()` to finalize and store the current article when a new header or article starts.
  - Iterates through the lines:
    - Detects **LIBRO** headers using `HEADER_PATTERNS['libro']`.
    - Detects **TITULO** headers using `HEADER_PATTERNS['titulo']`.
    - Detects **CAPITULO** headers and optionally captures the next line as chapter description.
    - Detects article headers using `ARTICULO_PATTERN` and accumulates the article text until the next header or article.
  - Returns a list of dictionaries, each representing an article with metadata and cleaned text.

---

Esta función segmenta el texto legal en partes estructuradas: libros, títulos, capítulos y artículos en detalle.

- `extract_articles(lines)`: Recibe una lista de líneas de texto.
  - Mantiene variables de contexto para el libro, título, capítulo y descripción del capítulo actual.
  - Utiliza la función auxiliar `flush_article()` para finalizar y almacenar el artículo actual cuando empieza un nuevo encabezado o artículo.
  - Itera sobre las líneas:
    - Detecta encabezados de **LIBRO** con `HEADER_PATTERNS['libro']`.
    - Detecta encabezados de **TITULO** con `HEADER_PATTERNS['titulo']`.
    - Detecta encabezados de **CAPITULO** y opcionalmente captura la siguiente línea como descripción del capítulo.
    - Detecta encabezados de artículo con `ARTICULO_PATTERN` y acumula el texto del artículo hasta el siguiente encabezado o artículo.
  - Devuelve una lista de diccionarios, cada uno representando un artículo con sus metadatos y texto limpio.


In [12]:
def extract_articles(lines):
    """Segmenta libros, títulos, capítulos y artículos en detalle."""
    # Contexto de encabezados
    current_libro = None
    current_libro_num = None
    current_titulo = None
    current_capitulo = None
    current_capitulo_num = None
    current_capitulo_desc = None

    # Segmentación de artículos
    articles = []
    current_article_num = None
    current_article_lines = []

    def flush_article():
        if current_article_num is None:
            return
        body = "\n".join(current_article_lines).strip()
        articles.append({
            'articulo_numero': int(current_article_num),
            'libro': current_libro.lower() if current_libro else None,
            'libro_numero': current_libro_num,
            'titulo': current_titulo.lower() if current_titulo else None,
            'capitulo': current_capitulo.lower() if current_capitulo else None,
            'capitulo_numero': current_capitulo_num,
            'capitulo_descripcion': current_capitulo_desc.lower() if current_capitulo_desc else None,
            'texto': body.lower().replace('\n', ''),
        })

    i = 0
    while i < len(lines):
        ln = lines[i]

        # Detectar LIBRO
        m_lib = HEADER_PATTERNS['libro'].match(ln)
        if m_lib:
            current_libro = f"LIBRO {m_lib.group(1).title()}"
            current_libro_num = ROMAN_MAP.get(m_lib.group(1).upper())
            i += 1
            continue

        # Detectar TITULO
        m_tit = HEADER_PATTERNS['titulo'].match(ln)
        if m_tit:
            current_titulo = f"TITULO {m_tit.group(1).title()}"
            i += 1
            continue

        # Detectar CAPITULO
        m_cap = HEADER_PATTERNS['capitulo'].match(ln)
        if m_cap:
            roman = m_cap.group(1)
            current_capitulo = f"CAPITULO {roman}"
            current_capitulo_num = roman_to_int(roman)
            next_desc = None
            if i + 1 < len(lines):
                nxt = lines[i + 1]
                if not (HEADER_PATTERNS['libro'].match(nxt) or HEADER_PATTERNS['titulo'].match(nxt) or HEADER_PATTERNS['capitulo'].match(nxt) or ARTICULO_PATTERN.match(nxt)):
                    next_desc = nxt
            current_capitulo_desc = next_desc
            i += 2 if next_desc else 1
            continue

        # Detectar inicio de Artículo
        m_art = ARTICULO_PATTERN.match(ln)
        if m_art:
            flush_article()
            current_article_num = m_art.group(1)
            current_article_lines = []
            i += 1
            while i < len(lines):
                nxt = lines[i]
                if (HEADER_PATTERNS['libro'].match(nxt) or HEADER_PATTERNS['titulo'].match(nxt) or
                    HEADER_PATTERNS['capitulo'].match(nxt) or ARTICULO_PATTERN.match(nxt)):
                    break
                current_article_lines.append(nxt)
                i += 1
            continue

        i += 1

    flush_article()
    return articles

# Full Law Text Parsing / Parseo Completo del Texto Legal

This function combines metadata extraction and article segmentation to parse the entire legal text:

- `parse_law_text(raw_text)`: Receives the raw text of a law.
  - Cleans the text using `fix_text()` to correct encoding issues.
  - Splits the text into non-empty lines.
  - Calls `extract_metadata(lines)` to extract law number, promulgation date, and publication date.
  - Calls `extract_articles(lines)` to segment books, titles, chapters, and articles.
  - Returns a dictionary containing `meta` (metadata) and `articulos` (list of structured articles).

---

Esta función combina la extracción de metadatos y la segmentación de artículos para parsear todo el texto legal:

- `parse_law_text(raw_text)`: Recibe el texto bruto de una ley.
  - Limpia el texto usando `fix_text()` para corregir problemas de codificación.
  - Divide el texto en líneas no vacías.
  - Llama a `extract_metadata(lines)` para extraer número de ley, fecha de promulgación y fecha de publicación.
  - Llama a `extract_articles(lines)` para segmentar libros, títulos, capítulos y artículos.
  - Devuelve un diccionario que contiene `meta` (metadatos) y `articulos` (lista de artículos estructurados).


In [13]:
def parse_law_text(raw_text):
    """Parsea el texto completo en metadatos y artículos."""
    text = fix_text(raw_text)
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    
    meta = extract_metadata(lines)
    articles = extract_articles(lines)

    return {
        'meta': meta,
        'articulos': articles,
    }

In [24]:
parsed = parse_law_text(texto_limpio)

# Preview Parsed Data / Previsualización de Datos Parseados

This code prints a preview of the parsed law data in JSON format:

- Uses `json.dumps` to convert a dictionary into a formatted JSON string.
- The dictionary contains:
  - `meta`: The metadata of the law.
  - `preview_articulos`: Only the first three articles for quick inspection.
- `ensure_ascii=False` preserves special characters.
- `indent=4` makes the JSON readable.

---

Este código imprime una previsualización de los datos parseados de la ley en formato JSON:

- Usa `json.dumps` para convertir un diccionario en una cadena JSON formateada.
- El diccionario contiene:
  - `meta`: Los metadatos de la ley.
  - `preview_articulos`: Solo los tres primeros artículos para una inspección rápida.
- `ensure_ascii=False` preserva los caracteres especiales.
- `indent=4` hace que el JSON sea legible.


In [23]:
print(json.dumps({
    'meta': parsed['meta'],
    'preview_articulos': parsed['articulos'][:3]
}, ensure_ascii=False, indent=4))

{
    "meta": {
        "numero_ley": "213",
        "fecha_promulgacion": "29-06-1993",
        "fecha_publicacion": "29-10-1993"
    },
    "preview_articulos": [
        {
            "articulo_numero": 1,
            "libro": "libro primero",
            "libro_numero": 1,
            "titulo": "titulo primero",
            "capitulo": "capitulo i",
            "capitulo_numero": 1,
            "capitulo_descripcion": "del objeto y aplicación del código",
            "texto": "este código tiene por objeto establecer normas para regular las relaciones entre los trabajadores y empleadores, concernientes a la prestación subordinada y retribuida de la actividad laboral."
        },
        {
            "articulo_numero": 2,
            "libro": "libro primero",
            "libro_numero": 1,
            "titulo": "titulo primero",
            "capitulo": "capitulo i",
            "capitulo_numero": 1,
            "capitulo_descripcion": "del objeto y aplicación del código",
          

# Save Parsed Data to JSON / Guardar Datos Parseados en JSON

This function saves the parsed law data into a JSON file:

- `save_parsed_json(parsed, filename, out_dir)`:
  - `parsed`: The dictionary containing `meta` and `articulos`.
  - `filename`: Name of the JSON file to create (default `"codigo_trabajo_articulos.json"`).
  - `out_dir`: Directory to save the file (default `"../data/processed"`).
- Ensures the output directory exists with `mkdir(parents=True, exist_ok=True)`.
- Writes the JSON data with `ensure_ascii=False` to preserve special characters and `indent=2` for readability.
- Prints the path of the saved file and the total number of articles.

---

Esta función guarda los datos parseados de la ley en un archivo JSON:

- `save_parsed_json(parsed, filename, out_dir)`:
  - `parsed`: Diccionario que contiene `meta` y `articulos`.
  - `filename`: Nombre del archivo JSON a crear (por defecto `"codigo_trabajo_articulos.json"`).
  - `out_dir`: Carpeta donde se guardará el archivo (por defecto `"../data/processed"`).
- Asegura que la carpeta de salida exista usando `mkdir(parents=True, exist_ok=True)`.
- Escribe los datos en JSON con `ensure_ascii=False` para preservar caracteres especiales y `indent=2` para legibilidad.
- Imprime la ruta del archivo guardado y el número total de artículos.


In [17]:
def save_parsed_json(parsed, filename="codigo_trabajo_articulos.json", out_dir="../data/processed"):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / filename

    with out_path.open("w", encoding="utf-8") as f:
        json.dump(parsed, f, ensure_ascii=False, indent=2)

    print(f"Guardado: {out_path}")
    print(f"Artículos totales: {len(parsed.get('articulos', []))}")

In [19]:
save_parsed_json(parsed)

Guardado: ../data/processed/codigo_trabajo_articulos.json
Artículos totales: 410
